In [1]:
import tensorflow as tf
import importlib.util
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
print("done")

done


# Check the normalisation by feeding with normal random values

In [2]:
arch_path = 'arch_invariant_c.py'
bands = 4

tf.reset_default_graph()
spec = importlib.util.spec_from_file_location("module.name", arch_path)
neural = importlib.util.module_from_spec(spec)
spec.loader.exec_module(neural)
CNN = neural.CNN
print("done")

cnn = CNN()
print("done")

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
print("done")

cnn.create_architecture()
print("done")

sess.run(tf.global_variables_initializer())
print("done")

done
done
done
[None, 1, 1, 128]
done
done


In [3]:
tensors = [o.inputs[0] for o in tf.get_default_graph().get_operations() if not 'gradients' in o.name 
           and ('Conv2D' in o.name or 'MatMul' in o.name or 'MaxPool' in o.name or 'Sigmoid' in o.name)]
tensors

[<tf.Tensor 'input:0' shape=(?, 424, 424, 3) dtype=float32>,
 <tf.Tensor 'input:0' shape=(?, 424, 424, 3) dtype=float32>,
 <tf.Tensor 'nn/bn-4/batchnorm/add_1:0' shape=(?, 211, 211, 4) dtype=float32>,
 <tf.Tensor 'nn/bn-4/batchnorm/add_1:0' shape=(?, 211, 211, 4) dtype=float32>,
 <tf.Tensor 'nn/bn-8x1/batchnorm/add_1:0' shape=(?, 211, 211, 8) dtype=float32>,
 <tf.Tensor 'nn/bn-8x1/batchnorm/add_1:0' shape=(?, 211, 211, 8) dtype=float32>,
 <tf.Tensor 'nn/convolution2_1/mul_2:0' shape=(?, 209, 209, 4) dtype=float32>,
 <tf.Tensor 'nn/convolution2_1/mul_2:0' shape=(?, 209, 209, 4) dtype=float32>,
 <tf.Tensor 'nn/convolution2_1/mul_3:0' shape=(?, 209, 209, 8) dtype=float32>,
 <tf.Tensor 'nn/convolution2_1/mul_3:0' shape=(?, 209, 209, 8) dtype=float32>,
 <tf.Tensor 'nn/bn-8/batchnorm/add_1:0' shape=(?, 103, 103, 8) dtype=float32>,
 <tf.Tensor 'nn/bn-8/batchnorm/add_1:0' shape=(?, 103, 103, 8) dtype=float32>,
 <tf.Tensor 'nn/bn-8x2/batchnorm/add_1:0' shape=(?, 103, 103, 16) dtype=float32>,
 <

In [4]:
x = np.random.normal(size=(10, 424, 424, 3))
y = sess.run(tensors, feed_dict={cnn.tfx: x})
print("done")

done


In [5]:
for no in range(3):
    x = np.random.normal(size=(10, 101, 101, bands))
    y_ = sess.run(tensors, feed_dict={cnn.tfx: x, cnn.tfkp: kp, cnn.tfacc: acc})
    y = [np.append(i, j, axis=0) for i,j in zip(y,y_)]
    print(no, end='... ', flush=True)

NameError: name 'kp' is not defined

In [6]:
[(np.prod(t.shape[1:]), t.shape, np.mean(t), np.std(t)) for t in y]

[(539328, (10, 424, 424, 3), -3.2674074e-05, 0.99978817),
 (539328, (10, 424, 424, 3), -3.2674074e-05, 0.99978817),
 (178084, (10, 211, 211, 4), 0.0070961006, 1.039415),
 (178084, (10, 211, 211, 4), 0.0070961006, 1.039415),
 (356168, (10, 211, 211, 8), -0.031331178, 0.95333815),
 (356168, (10, 211, 211, 8), -0.031331178, 0.95333815),
 (174724, (10, 209, 209, 4), 0.18069285, 1.2866882),
 (174724, (10, 209, 209, 4), 0.18069285, 1.2866882),
 (349448, (10, 209, 209, 8), 0.1518323, 1.180337),
 (349448, (10, 209, 209, 8), 0.1518323, 1.180337),
 (84872, (10, 103, 103, 8), 0.6186229, 1.9180104),
 (84872, (10, 103, 103, 8), 0.6186229, 1.9180104),
 (169744, (10, 103, 103, 16), 0.23557712, 1.4380996),
 (169744, (10, 103, 103, 16), 0.23557712, 1.4380996),
 (81608, (10, 101, 101, 8), 0.15463896, 1.9222602),
 (81608, (10, 101, 101, 8), 0.15463896, 1.9222602),
 (163216, (10, 101, 101, 16), -0.30847421, 0.87923884),
 (163216, (10, 101, 101, 16), -0.30847421, 0.87923884),
 (38416, (10, 49, 49, 16), 0.7

In [7]:
np.sum(y[11]==0.0) / y[11].size

0.0

## Normalisation propagation for pool max 2x2

In [ ]:
x = np.random.normal(size=(1000000, 4))
x = np.amax(x, 1)
m, s = (np.mean(x), np.std(x))
a, b = 1/s, m/s
(m,s),(a,b)

In [ ]:
x = np.random.normal(size=(1000000, 4))
x = a * x - b
x = np.amax(x, 1)
np.mean(x), np.std(x)